In [1]:
import time
import pandas as pd

## --- Day 10: Adapter Array ---
Patched into the aircraft's data port, you discover weather forecasts of a massive tropical storm. Before you can figure out whether it will impact your vacation plans, however, your device suddenly turns off!

Its battery is dead.

You'll need to plug it in. There's only one problem: the charging outlet near your seat produces the wrong number of jolts. Always prepared, you make a list of all of the joltage adapters in your bag.

Each of your joltage adapters is rated for a specific output joltage (your puzzle input). Any given adapter can take **an input 1, 2, or 3 jolts lower** than its rating and still produce its rated output joltage.

In addition, your device has a built-in joltage adapter rated for **3 jolts higher than the highest-rated **adapter in your bag. (If your adapter list were 3, 9, and 6, your device's built-in adapter would be rated for 12 jolts.)

Treat the charging outlet near your seat as having an effective joltage rating of 0.

Since you have some time to kill, you might as well test all of your adapters. Wouldn't want to get to your resort and realize you can't even charge your device!

If you use every adapter in your bag at once, what is the distribution of joltage differences between the charging outlet, the adapters, and your device?

Find a chain that uses all of your adapters to connect the charging outlet to your device's built-in adapter and count the joltage differences between the charging outlet, the adapters, and your device. **What is the number of 1-jolt differences multiplied by the number of 3-jolt differences?**

In [2]:
start_time = time.time()

with open('input-files/input-day10.txt', 'r') as fic:
    jolters = [int(x) for x in fic.read().splitlines()]

jolters.sort()
jolters = [0] + jolters + [jolters[-1] + 3]

gaps = [jolters[n + 1] - jolters[n] for n in range(len(jolters) - 1)]

gap1 = sum([gap == 1 for gap in gaps])
gap3 = sum([gap == 3 for gap in gaps])

print(gap1 * gap3)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

1856
Spent time: 0.001 s


### --- Part Two ---
To completely determine whether you have enough adapters, you'll need to figure out how many different ways they can be arranged. Every arrangement needs to connect the charging outlet to your device. The previous rules about when adapters can successfully connect still apply.

The first example above (the one that starts with 16, 10, 15) supports the following arrangements:

* (0), 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, (22)
* (0), 1, 4, 5, 6, 7, 10, 12, 15, 16, 19, (22)
* (0), 1, 4, 5, 7, 10, 11, 12, 15, 16, 19, (22)
* (0), 1, 4, 5, 7, 10, 12, 15, 16, 19, (22)
* (0), 1, 4, 6, 7, 10, 11, 12, 15, 16, 19, (22)
* (0), 1, 4, 6, 7, 10, 12, 15, 16, 19, (22)
* (0), 1, 4, 7, 10, 11, 12, 15, 16, 19, (22)
* (0), 1, 4, 7, 10, 12, 15, 16, 19, (22)

(The charging outlet and your device's built-in adapter are shown in parentheses.) Given the adapters from the first example, the total number of arrangements that connect the charging outlet to your device is 8.

What is **the total number of distinct ways you can arrange** the adapters to connect the charging outlet to your device?

In [3]:
def nb_possibilities(chain):
    if len(chain) < 2:
        return 1
    elif len(chain) == 2:
        if chain[1] - chain[0] <= 3:
            return 1
        return 0
    else:
        if chain[1] - chain[0] <= 3:
            return nb_possibilities(chain[1:]) + nb_possibilities([chain[0]] + chain[2:])
        return 0

In [4]:
start_time = time.time()

large_gaps = [n for n in range(len(gaps)) if gaps[n] == 3]
by = 6
n = by
total_possibilities = nb_possibilities(jolters[:(large_gaps[n] + 1)])
while n < (len(large_gaps) - by):
    total_possibilities *= nb_possibilities(jolters[large_gaps[n]:(large_gaps[n + by] + 1)])
    n += by
total_possibilities *= nb_possibilities(jolters[large_gaps[n]:])

print(total_possibilities)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

2314037239808
Spent time: 0.021 s


## --- Day 11: Seating System ---
Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either **floor (.)**, an **empty seat (L)**, or an **occupied seat (#)**. For example, the initial seat layout might look like this:

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the **number of occupied seats adjacent to a given sea**t (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are **applied to every seat simultaneously**:

* If a seat is **empty (L)** and there are **no occupied seats adjacent to it**, the seat becomes occupied.
* If a seat is **occupied (#)** and **four or more seats adjacent to it are also occupied**, the seat becomes empty.
* Otherwise, the seat's state does not change.
* Floor (.) never changes; seats don't move, and nobody sits on the floor.

Simulate your seating area by applying the seating rules repeatedly **until no seats change state**. How many seats end up occupied?

In [5]:
def visibles(area, lig, col):
    return (
        area[lig - 1][(col - 1):(col + 2)] + 
        area[lig][col - 1] + area[lig][col + 1] + 
        area[lig + 1][(col - 1):(col + 2)]
    )

In [6]:
def remplace(text, position, by):
    tempo = list(text)
    tempo[position] = by
    return ''.join(tempo)

In [7]:
def next_step(area, tolerance):
    ligs = len(area)
    cols = len(area[0])
    borded = ['.' * (cols + 2)] + ['.' + x + '.' for x in area] + ['.' * (cols + 2)]
    new = borded.copy()
    for lig in range(1, ligs + 1):
        for col in range(1, cols + 1):
            if borded[lig][col] == 'L':
                borders = visibles(borded, lig, col)
                if ('#' not in borders):
                    new[lig] = remplace(new[lig], col, '#')
            elif borded[lig][col] == '#':
                borders = visibles(borded, lig, col)
                if (borders.count('#') >= tolerance):
                    new[lig] = remplace(new[lig], col, 'L')
    return [x[1:-1] for x in new[1:-1]]

In [8]:
start_time = time.time()

with open('input-files/input-day11.txt', 'r') as fic:
    area = fic.read().splitlines()

previous = ['.' * len(x) for x in area]
new = area
while previous != new:
    previous = new
    new = next_step(previous, 4)

print(sum([x.count('#') for x in new]))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

2277
Spent time: 0.640 s


### --- Part Two ---
As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider **the first seat in each of those eight directions**. 

Also, people seem to be more tolerant than you expected: it now takes **five or more visible occupied seats for an occupied seat to become empty** (rather than four or more from the previous rules). The **other rules still apply**: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

In [9]:
def visibles(area, lig, col):
    ligs = len(area)
    cols = len(area[0])
    seen = ''
    for NS in range(-1, 2):
        lenV = int((1 - ligs) / 2 * NS * NS + (ligs / 2 - lig - 1/2) * NS + ligs)
        for WE in range(-1, 2):
            lenH = int((1 - cols) / 2 * WE * WE + (cols / 2 - col - 1/2) * WE + cols)
            if (NS != 0) | (WE != 0):
                n = 1
                neighbour = '.'
                while (n < min(lenV, lenH)) & (neighbour == '.'):
                    neighbour = area[lig + NS * n][col + WE * n]
                    n += 1
                seen += neighbour
    return seen

In [10]:
start_time = time.time()

previous = ['.' * len(x) for x in area]
new = area
while previous != new:
    previous = new
    new = next_step(previous, 5)

print(sum([x.count('#') for x in new]))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

2066
Spent time: 8.275 s


## --- Day 12: Rain Risk ---
Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

* Action N means **to move north** by the given value.
* Action S means **to move south** by the given value.
* Action E means **to move east** by the given value.
* Action W means **to move west** by the given value.
* Action L means **to turn left** the given number of degrees.
* Action R means **to turn right** the given number of degrees.
* Action F means **to move forward** by the given value in the direction the ship is currently facing.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [11]:
def action(ship, instruction):
    if instruction.startswith('N'):
        ship['NS'] += int(instruction[1:])
    elif instruction.startswith('S'):
        ship['NS'] -= int(instruction[1:])
    elif instruction.startswith('E'):
        ship['WE'] += int(instruction[1:])
    elif instruction.startswith('W'):
        ship['WE'] -= int(instruction[1:])
    elif instruction.startswith('F'):
        ship = action(ship, ship['facing'] + instruction[1:])
    elif instruction.startswith('R'):
        right_turn = 'ESWNESWN'
        ship['facing'] = right_turn[right_turn.find(ship['facing']) + int(int(instruction[1:]) / 90)]
    elif instruction.startswith('L'):
        left_turn = 'ENWSENWS'
        ship['facing'] = left_turn[left_turn.find(ship['facing']) + int(int(instruction[1:]) / 90)]
    return ship

In [12]:
start_time = time.time()

with open('input-files/input-day12.txt', 'r') as fic:
    instructions = fic.read().rstrip('\n').split('\n')
    instructions

ship = {'NS': 0, 'WE': 0, 'facing': 'E'}
for instruction in instructions:
    ship = action(ship, instruction)

print(abs(ship['NS']) + abs(ship['WE']))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

796
Spent time: 0.001 s


### --- Part Two ---
Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to **move a waypoint which is relative to the ship's position**:

* Action N means to move the waypoint north by the given value.
* Action S means to move the waypoint south by the given value.
* Action E means to move the waypoint east by the given value.
* Action W means to move the waypoint west by the given value.
* Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
* Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
* Action F means **to move forward to the waypoint** a number of times equal to the given value.

The **waypoint starts 10 units east and 1 unit north** relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [13]:
def ship_action(ship, waypoint, instruction):
    if instruction.startswith('F'):
        ship['NS'] += waypoint['NS'] * int(instruction[1:])
        ship['WE'] += waypoint['WE'] * int(instruction[1:])
    return ship

In [14]:
def waypoint_action(waypoint, instruction):
    if instruction.startswith('N'):
        waypoint['NS'] += int(instruction[1:])
    elif instruction.startswith('S'):
        waypoint['NS'] -= int(instruction[1:])
    elif instruction.startswith('E'):
        waypoint['WE'] += int(instruction[1:])
    elif instruction.startswith('W'):
        waypoint['WE'] -= int(instruction[1:])
    elif instruction.startswith('R') | instruction.startswith('L'):
        if instruction.startswith('R'):
            turn = 1
        else:
            turn = - 1
        for n in range(int(int(instruction[1:]) / 90)):
            waypoint['WE'], waypoint['NS'] = turn * waypoint['NS'], - turn * waypoint['WE']
    return waypoint

In [15]:
start_time = time.time()

ship = {'NS': 0, 'WE': 0}
waypoint = {'NS': 1, 'WE': 10}
for instruction in instructions:
    ship = ship_action(ship, waypoint, instruction)
    waypoint = waypoint_action(waypoint, instruction)

print(ship)

print(abs(ship['NS']) + abs(ship['WE']))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

{'NS': -15029, 'WE': -24417}
39446
Spent time: 0.002 s


## --- Day 13: Shuttle Search ---
Your ferry can make it safely to a nearby port, but it won't get much further. When you call to book another ship, you discover that no ships embark from that port to your vacation island. You'll need to get from the port to the nearest airport.

Fortunately, a shuttle bus service is available to bring you from the sea port to the airport! **Each bus has an ID number that also indicates how often the bus leaves for the airport**.

Bus schedules are defined based on a timestamp that measures the number of minutes since some fixed reference point in the past. **At timestamp 0, every bus simultaneously departed from the sea port**. After that, each bus travels to the airport, then various other locations, and finally returns to the sea port to repeat its journey forever.

Your notes (your puzzle input) consist of two lines. 
* The first line is your **estimate of the earliest timestamp you could depart on a bus**. 
* The second line lists **the bus IDs that are in service** according to the shuttle company; entries that show **x must be out of service**, so you decide to ignore them.

To save time once you arrive, your goal is to figure out the earliest bus you can take to the airport. (There will be exactly one such bus.)


What is the **ID of the earliest bus you can take to the airport multiplied by the number of minutes you'll need to wait** for that bus?

In [16]:
start_time = time.time()

with open('input-files/input-day13.txt', 'r') as fic:
    flyer = fic.read().rstrip('\n').split('\n')

arrival = int(flyer[0])
buses = [int(x) for x in flyer[1].split(',') if x != 'x']

schedule = (0, max(buses) + 1)
for bus in buses:
    waiting_time = (arrival // bus + 1) * bus - arrival
#     print(f'Bus {bus}, attente {waiting_time}')
    if waiting_time < schedule[1]:
        schedule = (bus, waiting_time)

print(schedule[0] * schedule[1])

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

4207
Spent time: 0.001 s


### --- Part Two ---
The shuttle company is running a contest: one gold coin for anyone that can find the **earliest timestamp** such that the **first bus ID departs at that time** and **each subsequent listed bus ID departs at that subsequent minute** (position in the list). (The first line in your input is no longer relevant.)

What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?

In [17]:
def prochain_depart(buses, offsets):
    departure = buses[0] - offsets[0]
    nb = len(buses)
    go_on = True
    while go_on: # à chaque boucle on tente une nouvelle heure de départ
        b = 0 # position dans la liste des bus (et des offsets)
        possible = True
        while (b < nb) & possible: # on se fait tous les bus tant que l'horaire est compatible avec eux
            attente = (buses[b] - departure % buses[b]) % buses[b]
            if attente != offsets[b]:
                possible = False
                departure += buses[0]
            else:
                b += 1
        if b == nb: # si on a fait tous les bus, c'est qu'ils sont tous compatibles, on arrête
            go_on = False
    return departure

In [18]:
start_time = time.time()

buses_list = flyer[1].split(',')
buses = [int(x) for x in buses_list if x != 'x']
offsets = [(n % int(buses_list[n])) for n in range(len(buses_list)) if buses_list[n] != 'x']

while len(buses) > 2:
    depart = prochain_depart(buses[:2], offsets[:2])
    fake_bus = buses[0] * buses[1]
    fake_offset = fake_bus - prochain_depart(buses[:2], offsets[:2])
    buses = [fake_bus] + buses[2:]
    offsets = [fake_offset] + offsets[2:]
print(f'Next in minutes: {prochain_depart(buses, offsets)}')
print(f'  which is, in years: {prochain_depart(buses, offsets) / 60 / 24 / 365.25:0.0f}')

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

Next in minutes: 725850285300475
  which is, in years: 1380048455
Spent time: 0.001 s


## --- Day 14: Docking Data ---
As your ferry approaches the sea port, the captain asks for your help again. The computer system that runs this port isn't compatible with the docking program on the ferry, so the docking parameters aren't being correctly initialized in the docking program's memory.

After a brief inspection, you discover that the sea port's computer system uses a strange bitmask system in its initialization program. Although you don't have the correct decoder chip handy, you can emulate it in software!

The initialization program (your puzzle input) can either **update the bitmask** or **write a value to memory**. Values and memory addresses are both 36-bit unsigned integers. For example, ignoring bitmasks for a moment, a line like **mem[8] = 11 would write the value 11 to memory address 8**.

The bitmask is always given as a string of 36 bits, written with the most significant bit (representing 2^35) on the left and the least significant bit (2^0, that is, the 1s bit) on the right. The current bitmask is applied to values immediately before they are written to memory: a **0 or 1 overwrites** the corresponding bit in the value, while an **X leaves the bit** in the value unchanged.

To initialize your ferry's docking program, you need the sum of all values left in memory after the initialization program completes.
Execute the initialization program. What is the sum of all values left in memory after it completes?

In [19]:
def setmem(memory, instruction, mask):
    place = int(instruction[(instruction.find('[') + 1): instruction.find(']')])
    value = int(instruction[(instruction.find('=') + 1):])
    text_value = bin(value)[2:]
    text_value = '0' * (36 - len(text_value)) + text_value
    new_value = ''.join([(m if m != 'X' else v) for (v, m) in zip(text_value, mask)])
    memory[place] = new_value
    return memory

In [20]:
start_time = time.time()

with open('input-files/input-day14.txt', 'r') as fic:
    instructions = fic.read().rstrip('\n').split('\n')

mask = 'X' * 36
memory = (['0' * 36] * 
          (max([int(x[(x.find('[') + 1): x.find(']')]) for x in instructions if x.startswith('mem')]) + 1))

for n in range(len(instructions)):
    if instructions[n].startswith('mem'):
        memory = setmem(memory, instructions[n], mask)
    elif instructions[n].startswith('mask'):
        mask = instructions[n][(instructions[n].find('=') + 2):]

print(sum([int(x, 2) for x in memory]))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

10717676595607
Spent time: 0.023 s


## --- Part Two ---

For some reason, the sea port's computer system still can't communicate with your ferry's docking program. It must be using version 2 of the decoder chip!

A version 2 decoder chip doesn't modify the values being written at all. Instead, it acts as a **memory address decoder**. Immediately before a value is written to memory, each bit in the bitmask modifies the corresponding bit of the destination memory address in the following way:

* If the bitmask bit is **0**, the corresponding memory address bit is **unchanged**.
* If the bitmask bit is **1**, the corresponding memory address bit is **overwritten with 1**.
* If the bitmask bit is **X**, the corresponding memory address bit is **floating**. A floating bit is not connected to anything and instead fluctuates unpredictably. In practice, this means the floating bits will **take on all possible values**, potentially causing many memory addresses to be written all at once!

Execute the initialization program using an emulator for a version 2 decoder chip. What is the sum of all values left in memory after it completes?

In [21]:
def apply_mask(mask, value):
    adresses = ['']
    for n in range(36):
        if mask[n] == '0':
            adresses = [x + value[n] for x in adresses]
        elif mask[n] == '1':
            adresses = [x + '1' for x in adresses]
        elif mask[n] == 'X':
            adresses = [x + '0' for x in adresses] + [x + '1' for x in adresses]
    return [int(x, 2) for x in adresses]

In [22]:
def setmem(memory, instruction, mask):
    place = int(instruction[(instruction.find('[') + 1): instruction.find(']')])
    value = int(instruction[(instruction.find('=') + 1):])
    text_place = bin(place)[2:]
    text_place = '0' * (36 - len(text_place)) + text_place
    adresses = apply_mask(mask, text_place)
    for a in adresses:
        memory[a] = value
    return memory

In [23]:
start_time = time.time()

mask = '0' * 36
memory = dict()

for n in range(len(instructions)):
    if instructions[n].startswith('mem'):
        memory = setmem(memory, instructions[n], mask)
    elif instructions[n].startswith('mask'):
        mask = instructions[n][(instructions[n].find('=') + 2):]

print(sum(memory.values()))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

3974538275659
Spent time: 0.108 s


## --- Day 15: Rambunctious Recitation ---
You catch the airport shuttle and try to book a new flight to your vacation island. Due to the storm, all direct flights have been cancelled, but a route is available to get around the storm. You take it.

While you wait for your flight, you decide to check in with the Elves back at the North Pole. They're playing a memory game and are ever so excited to explain the rules!

In this game, the players take turns saying numbers. They begin by taking turns reading from a list of starting numbers (your puzzle input). Then, each turn consists of considering the most recently spoken number:

* If that was the **first time** the number has been spoken, the current player says **0**.
* Otherwise, the number had been spoken before; the current player announces **how many turns apart** the number is from when it was previously spoken.

So, after the starting numbers, each turn results in that player speaking aloud either 0 (if the last number is new) or an age (if the last number is a repeat).

Their question for you is: what will be the 2020th number spoken?

In [24]:
start_time = time.time()

numbers = [9, 3, 1, 0, 8, 4]
start = len(numbers) - 1
for n in range(start, 2020 - 1):
    if (numbers[n] in numbers[:-1]):
        numbers += [numbers[::-1][1:].index(numbers[n]) + 1]
    else:
        numbers += [0]
print(numbers[-1])

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

371
Spent time: 0.023 s


### --- Part Two ---
Impressed, the Elves issue you a challenge: determine the 30000000th number spoken.

In [25]:
start_time = time.time()

numbers = [9, 3, 1, 0, 8, 4]
start = len(numbers) - 1
last = {numbers[n]: n for n in range(len(numbers) - 1)}
for n in range(start, 30000000 - 1):
    if (numbers[n] in last.keys()):
        numbers += [n - last[numbers[n]]]
    else:
        numbers += [0]
    last[numbers[n]] = n
print(numbers[-1])

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

352
Spent time: 25.155 s


## --- Day 16: Ticket Translation ---
As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a **rule like class: 1-3 or 5-7** means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a **single line of comma-separated values**. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

Start by determining which tickets are **completely invalid; these are tickets that contain values which aren't valid for any field**. Ignore your ticket for now.

Adding together all of the invalid values produces your ticket **scanning error rate**.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?

In [26]:
def extract_numbers(rule):
    flatten = [int(y) for x in [x.split('or') for x in rule.split('-')] for y in x]
    ok = []
    for n in range(int(len(flatten) / 2)):
        ok += [i for i in range(flatten[2 * n], flatten[2 * n + 1] + 1)]
    return ok

In [27]:
start_time = time.time()

with open('input-files/input-day16.txt', 'r') as fic:
    inputs = fic.read().rstrip('\n').split('\n\n')

rules = {rule.split(':')[0]: rule.split(':')[1].strip() for rule in inputs[0].split('\n')}
numrules = {key: extract_numbers(rules[key]) for key in rules}

my_ticket = [int(x) for x in inputs[1].split('\n')[1].split(',')]

nearby_tickets = [[int(x) for x in ticket.split(',')] for ticket in inputs[2].split('\n')[1:]]

all_valid_numbers = list(set([x for l in numrules.values() for x in l]))

scanning_error = 0
valid_tickets = []
for neighbour in nearby_tickets:
    nok_numbers = sum([x for x in neighbour if x not in all_valid_numbers])
    scanning_error += nok_numbers
    if nok_numbers == 0:
        valid_tickets += [neighbour]

print(scanning_error)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

27870
Spent time: 0.031 s


### --- Part Two ---
Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, **determine what order the fields appear on the tickets**. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

In [28]:
start_time = time.time()

valid_tickets += [my_ticket]

df = pd.DataFrame(valid_tickets)

first_guesses = {}
for col in df.columns:
    for key in numrules.keys():
        if df[col].isin(numrules[key]).mean() == 1:
            if key in first_guesses.keys():
                first_guesses[key] += [col]
            else:
                first_guesses[key] = [col]

guesses = dict.fromkeys(first_guesses.keys(), -1)
previous_guesses = first_guesses.copy()
new_guesses = previous_guesses.copy()

while min(guesses.values()) == -1:
    for k, v in previous_guesses.items():
        if len(v) == 1:
            guesses[k] = v[0]
            del new_guesses[k]
            for k0 in new_guesses:
                new_guesses[k0] = [x for x in new_guesses[k0] if x != v[0]]
    previous_guesses = new_guesses.copy()

departure_values = [my_ticket[n] for n in [v for k, v in guesses.items() if k.startswith('departure')]]

result = 1
for n in departure_values:
    result *= n

print(result)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

3173135507987
Spent time: 0.133 s


## --- Day 17: Conway Cubes ---
As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an **infinite 3-dimensional grid**. At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified **active (#)** or **inactive (.)** state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers **its neighbors: any of the 26 other cubes** where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

During a cycle, all cubes **simultaneously** change their state according to the following rules:

* If a cube is **active** and **exactly 2 or 3** of its neighbors are also active, the cube **remains active**. Otherwise, the cube becomes inactive.
* If a cube is **inactive** but **exactly 3** of its neighbors are active, the cube **becomes active**. Otherwise, the cube remains inactive.

The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the **six-cycle boot process**.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?

In [29]:
start_time = time.time()

with open('input-files/input-day17.txt', 'r') as fic:
    initial = fic.read().rstrip('\n').split('\n')

df = pd.DataFrame(columns=['x', 'y', 'z', 'active'])
for x in range(len(initial)):
    for y in range(len(initial[x])):
        if initial[x][y] == '#':
            df = df.append(pd.Series({'x': x, 'y': y, 'z': 0, 'active': True}), ignore_index=True)

previous_step = df.copy()
for loop in range(6):
    neighbours = pd.DataFrame(columns=['x', 'y', 'z'])
    
    for n in previous_step.index:
        activated = previous_step.loc[n]
        x = [x + activated['x'] for x in range(-1, 2) for y in range(-1, 2) for z in range(-1, 2)]
        y = [y + activated['y'] for x in range(-1, 2) for y in range(-1, 2) for z in range(-1, 2)]
        z = [z + activated['z'] for x in range(-1, 2) for y in range(-1, 2) for z in range(-1, 2)]
        neighbours = neighbours.append(pd.DataFrame({'x': x, 'y': y, 'z': z, 'count': 0}))
        
    counts = neighbours.groupby(['x', 'y', 'z'], as_index=False).count()
    neighbours = counts.merge(previous_step, on=['x', 'y', 'z'], how='left', validate='1:1')
    
    neighbours['active'] = neighbours['active'].fillna(False)
    neighbours['count'] = neighbours['count'] - neighbours['active']
    
    next_step = neighbours.loc[(neighbours['count'] == 3) | 
                               ((neighbours['count'] == 2) & (neighbours['active']))]
    previous_step = next_step[['x', 'y', 'z', 'active']].copy()
    previous_step['active'] = True
    print(f'--- {previous_step.shape[0]:4.0f} ---')

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

---   77 ---
---   67 ---
---  148 ---
---  186 ---
---  294 ---
---  304 ---
Spent time: 2.786 s


## Part 2

Finalement, c'est en 4 dimensions

In [30]:
start_time = time.time()

df = pd.DataFrame(columns=['x', 'y', 'z', 'w', 'active'])
for x in range(len(initial)):
    for y in range(len(initial[x])):
        if initial[x][y] == '#':
            df = df.append(pd.Series({'x': x, 'y': y, 'z': 0, 'w': 0, 'active': True}), ignore_index=True)

previous_step = df.copy()
for loop in range(6):
    neighbours = pd.DataFrame(columns=['x', 'y', 'z', 'w'])
    
    for n in previous_step.index:
        activated = previous_step.loc[n]
        x = [x + activated['x'] for x in range(-1, 2) for y in range(-1, 2) for z in range(-1, 2) for w in range(-1, 2)]
        y = [y + activated['y'] for x in range(-1, 2) for y in range(-1, 2) for z in range(-1, 2) for w in range(-1, 2)]
        z = [z + activated['z'] for x in range(-1, 2) for y in range(-1, 2) for z in range(-1, 2) for w in range(-1, 2)]
        w = [w + activated['w'] for x in range(-1, 2) for y in range(-1, 2) for z in range(-1, 2) for w in range(-1, 2)]
        neighbours = neighbours.append(pd.DataFrame({'x': x, 'y': y, 'z': z, 'w': w, 'count': 0}))
        
    counts = neighbours.groupby(['x', 'y', 'z', 'w'], as_index=False).count()
    neighbours = counts.merge(previous_step, on=['x', 'y', 'z', 'w'], how='left', validate='1:1')
    
    neighbours['active'] = neighbours['active'].fillna(False)
    neighbours['count'] = neighbours['count'] - neighbours['active']
    
    next_step = neighbours.loc[(neighbours['count'] == 3) | 
                               ((neighbours['count'] == 2) & (neighbours['active']))]
    previous_step = next_step[['x', 'y', 'z', 'w', 'active']].copy()
    previous_step['active'] = True
    print(f'--- {previous_step.shape[0]:4.0f} ---')

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

---  209 ---
---  185 ---
---  892 ---
---  556 ---
--- 2824 ---
--- 1868 ---
Spent time: 28.163 s


## --- Day 18: Operation Order ---

As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework.

Unfortunately, it seems like this "math" follows different rules than you remember.

The homework (your puzzle input) consists of a series of expressions that consist of **addition (+)**, **multiplication (\*)**, and **parentheses ((...))**. Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, **the operators have the same precedence, and are evaluated left-to-right** regardless of the order in which they appear.

Before you can help with the homework, you need to understand it yourself. Evaluate the expression on each line of the homework; what is the sum of the resulting values?

In [31]:
def transform_parenthesis(formula):
    if ')' in formula:
        close = formula.index(')')
        left = formula[:close]
        left.reverse()
        start = len(left) - left.index('(')
        return transform_parenthesis(formula[:(start - 1)] + [formula[start:close]] + formula[(close + 1):])
    else:
        return formula

In [32]:
def solve_formula(formula):
    if type(formula) == str:
        return int(formula)
    elif len(formula) == 1:
        return solve_formula(formula[0])
    elif formula[-2] == '*':
        return solve_formula(formula[:-2]) * solve_formula(formula[-1])
    elif formula[-2] == '+':
        return solve_formula(formula[:-2]) + solve_formula(formula[-1])

In [33]:
start_time = time.time()

with open('input-files/input-day18.txt', 'r') as fic:
    operations = fic.read().rstrip('\n').split('\n')

result = sum([solve_formula(transform_parenthesis([x for x in list(operation) if x != ' '])) 
     for operation in operations])
print(result)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

12918250417632
Spent time: 0.008 s


### --- Part Two ---
You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: advanced math.

Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, **addition is evaluated before multiplication**.

What do you get if you add up the results of evaluating the homework problems using these new rules?

In [34]:
def solve_formula(formula):
    if type(formula) == str:
        return int(formula)
    elif len(formula) == 1:
        return solve_formula(formula[0])
    elif '*' in formula:
        times = formula.index('*')
        return solve_formula(formula[:times]) * solve_formula(formula[(times + 1):])
    elif formula[-2] == '+':
        return solve_formula(formula[:-2]) + solve_formula(formula[-1])

In [35]:
start_time = time.time()

result = sum([solve_formula(transform_parenthesis([x for x in list(operation) if x != ' '])) 
     for operation in operations])
print(result)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

171259538712010
Spent time: 0.011 s
